# Eve hands-on day 2

You start from a high level IR representing the following DSL

```python
def fun(out_field, in_field):
    out_field = -4 * in_field[0,0] + in_field[-1,0] + in_field[1,0] + in_field[0,-1] + in_field[0,1]
```

- Write a pass: extent analysis
- Lower from HIR to LIR

The HIR is provided in the file `hir.py` as well as the implementation of the Laplacian in that dialect (see below).
Feel free to change the skeleton for `ExtentAnalysis` and `HIRtoLIR` as you like.

## Preparation

Create a new file called `lir.py` and put your dialect from *Day 1* into that file. For simplicity also put the code generator in this file.

In [ ]:
# Definition of the Laplacian in HIR (don't change)

import hir
from hir import *

lap_expr = BinaryOp(left=BinaryOp(left=Literal(value="-4"), right=FieldAccess(name="in", offset=Offset.zero()), op="*"), right=BinaryOp(left=BinaryOp(left=FieldAccess(name="in", offset=Offset(i=-1,j=0)),right=FieldAccess(name="in", offset=Offset(i=1,j=0)),op="+"),right=BinaryOp(left=FieldAccess(name="in", offset=Offset(i=0,j=-1)),right=FieldAccess(name="in", offset=Offset(i=0,j=1)),op="+"),op="+"), op="+")
assign = AssignStmt(left=FieldAccess(name="out", offset=Offset.zero()), right=lap_expr)
stencil = Stencil(name="lap", params=[FieldParam(name="out"), FieldParam(name="in")], body=[assign])

from devtools import debug
debug(stencil)

In [ ]:
# Feel free to change the signature of the provided methods (or remove them) if you prefer a different structure.
import eve

class Extent(eve.Model):
    i_left: int
    i_right: int
    j_left: int
    j_right: int

    @classmethod
    def zero(cls):
        return cls(i_left = 0, i_right = 0, j_left = 0, j_right = 0)

    def __add__(self, other):
        if other is None:
            return self
        else:
            return type(self)(i_left=min(self.i_left, other.i_left), i_right=max(self.i_right, other.i_right),j_left=min(self.j_left, other.j_left), j_right=max(self.j_right, other.j_right))

    @classmethod
    def from_offset(cls, offset: Offset):
        return cls(i_left=min(0, offset.i), i_right=max(0,offset.i), j_left=min(0,offset.j), j_right=max(0, offset.j))

# class ExtentAnalysis(eve.NodeVisitor):
#     def __init__(self, **kwargs):
#         super().__init__()
#         self.extents = {}

#     @classmethod
#     def apply(cls, root: Stencil, **kwargs):
#         analysis = cls()
#         analysis.visit(root)
#         return analysis.extents

#     def visit_FieldAccess(self, node: FieldAccess, **kwargs):
#         if not kwargs["cur_assign"]:
#             raise ValueError("FieldAccess outside of assignment")
#         self.extents[kwargs["cur_assign"]] += Extent.from_offset(node.offset)

#     def visit_AssignStmt(self, node: AssignStmt, **kwargs):
#         self.extents[node.id_] = Extent.zero()
#         self.visit(node.right, cur_assign = node.id_)


class ExtentAnalysis(eve.NodeVisitor):
    @classmethod
    def apply(cls, root: Stencil, **kwargs):
        return cls().visit(root)
        
    def visit_Expr(self, node: Expr, **kwargs):
        extent = Extent.zero()
        for value in node.iter_children_values():
            extent += self.visit(value, **kwargs)
        return extent

    def visit_FieldAccess(self, node: FieldAccess, **kwargs):
        return Extent.from_offset(node.offset)

    def visit_AssignStmt(self, node: AssignStmt, **kwargs):
        return self.visit(node.right, **kwargs)
    
    def visit_Stencil(self, node: Stencil, **kwargs):
        return {s.id_: self.visit(s, **kwargs) for s in node.body}


In [ ]:
# Feel free to change the signature of the provided methods (or remove them) if you prefer a different structure.

import eve
import lir

class HIRToLIR(eve.NodeTranslator):
    def __init__(self, extents, **kwargs):
        self.extents = extents

    @classmethod
    def apply(cls, root: Stencil, extents, **kwargs):
        hir_to_lir = cls(extents)
        return hir_to_lir.visit(root)

    def _literal_translation(self, node: Node, **kwargs):
        new_class_name = node.__class__.__name__  # mapping[node.__class__]
        new_class = getattr(lir, new_class_name)
        return new_class(**{key: self.visit(value) for key, value in node.iter_children()})

    visit_Literal = _literal_translation
    visit_BinaryOp = _literal_translation
    visit_FieldAccess = _literal_translation
    visit_Offset = _literal_translation

    # def visit_Literal(self, node: Literal, **kwargs):
    #     return lir.Literal(value=node.value)

    # def visit_BinaryOp(self, node: BinaryOp, **kwargs):
    #     return lir.BinaryOp(left=self.visit(node.left), right=self.visit(node.right), op=node.op)
    
    # def visit_FieldAccess(self, node: FieldAccess, **kwargs):
    #     return lir.FieldAccess(name=node.name, offset=self.visit(node.offset))
    
    # def visit_Offset(self, node: Offset, **kwargs):
    #     return lir.Offset(i=node.i, j=node.j)

    def visit_FieldParam(self, node: FieldParam, **kwargs):
        return lir.FieldDecl(name=node.name)

    def visit_AssignStmt(self, node: AssignStmt, **kwargs):
        extent : Extent = self.extents[node.id_]
        return lir.HorizontalLoop(body=[lir.AssignStmt(left=self.visit(node.left), right=self.visit(node.right))], i_indent=lir.Indent(left=-extent.i_left, right=extent.i_right), j_indent=lir.Indent(left=-extent.j_left, right=extent.j_right))

    def visit_Stencil(self, node: Stencil, **kwargs):
        return lir.Fun(name=node.name, params=[self.visit(p) for p in node.params], horizontal_loops=[self.visit(stmt) for stmt in node.body])


In [ ]:
# No need to change anything here (except if you changed names of passes or signature of apply)

extents = ExtentAnalysis.apply(stencil)
lir_stencil = HIRToLIR.apply(stencil, extents)
debug(lir_stencil)

In [ ]:
# Don't touch
import os

cppcode = lir.LIR_to_cpp.apply(lir_stencil)
formatted_code = eve.codegen.format_source("cpp", cppcode, style="LLVM")
print(formatted_code)

output_file = "generated.hpp"
with open(output_file, "w+") as output:
    output.write(formatted_code)

In [ ]:
!g++ driver.cc -o lap
!./lap